## Shell for creating FastAPI for broswer url tester


In [ ]:
# stimutated browser history
browser_history =[
    "www.google.com",
    "www.facebook.com",
    "https://mail.google.com/mail/u/0/#inbox/FMfcgzQbgJRrhvBHMkvhWsgqLVVlHfst",
    "https://colab.research.google.com/drive/1ZUMV9KmgyMs6lZ1kMdPaZy279ylU-odI",
    "http://ggu.wifi/login?dst=http%3A%2F%2Fwww.msftconnecttest.com%2Fredirect",
    "xvideos.com",
    "pornhub.com",
    "xnxx.com",
    "https://www.kaggle.com/datasets?search=+LAION-5B+%28with+filtered+NSFW+tags%29",
    "https://drive.google.com/drive/my-drive",
    "www.gndivideos.com",

]

In [ ]:
# list of known adult key words
adult_keywords = ["porn", "sex", "xvideos", "xnxx", "redtube", "youporn", "hot", "nude", "xxx", "nsfw", "18+", "erotic","gndivideos"]
#making a function to check for adult keywords
def is_adult_url(url):
    return any(keyword in url.lower() for keyword in adult_keywords)
# scan full history
flagged_sites = [url for url in browser_history if is_adult_url(url)]
flagged_sites

['xvideos.com',
 'pornhub.com',
 'xnxx.com',
 'https://www.kaggle.com/datasets?search=+LAION-5B+%28with+filtered+NSFW+tags%29',
 'www.gndivideos.com']

## creation of Fast api


In [ ]:
!pip install fastapi nest-asyncio unicorn pyngrok --quiet

In [6]:
import nest_asyncio
from fastapi import FastAPI, Request
from fastapi.responses import JSONResponse
import uvicorn
from pyngrok import ngrok
import threading
ngrok.set_auth_token("2vOoQ2J2A8Go6bksZehPy1SB6CW_FDLfYpkAFgJNoRgmBoHx")
# saving the FastAPI function inside app object
app = FastAPI()
@app.get('/')
async def root():
  return{"messsage":"helloooooo"}

# creating a route to check
@app.post("/check")
# creating a asynnc function
async def check_url(request: Request):
# saving the json response in data variable
 data = await request.json()
#  get the url
 url = data.get("url")
#  check whether the url is adult wala
 is_adult = is_adult_url(url)
 return JSONResponse({"url":url, "is_adult":is_adult})

## launch FastAPI server in collab

In [7]:
nest_asyncio.apply()
# creating the public url
public_url = ngrok.connect(8000)
print(f"Public FastAPI: {public_url}")
# running the fastAPI
def run():
    uvicorn.run(app, host="0.0.0.0", port=8000)

threading.Thread(target=run).start()

Public FastAPI: NgrokTunnel: "https://6294e6ebe92a.ngrok-free.app" -> "http://localhost:8000"


## shell for browser level url integration

In [8]:
!pip install browser_history
from browser_history import get_history
import requests
import time
# get history from chrome
outputs = get_history()
history = outputs.histories #list of histories with date and time
# getting the latest 10 urls
latest_urls = [url for _, url in sorted(history,reverse=True)[:10]]
latest_urls += [
    "https://www.redtube.com",
    "https://example.com/normal-page",
    "https://xvideos.com"
]
print("🔍 Last 10 visited URLs:")
for i, url in enumerate(latest_urls,1):
 print(f"{i}.{url}")

INFO: Chromium browser is not installed
INFO:browser-history:Chromium browser is not installed
INFO: Chrome browser is not installed
INFO:browser-history:Chrome browser is not installed
INFO: Edge browser is not installed
INFO:browser-history:Edge browser is not installed
INFO: Opera browser is not installed
INFO:browser-history:Opera browser is not installed
INFO: OperaGX browser is not supported
INFO:browser-history:OperaGX browser is not supported
INFO: Brave browser is not installed
INFO:browser-history:Brave browser is not installed
INFO: Vivaldi browser is not installed
INFO:browser-history:Vivaldi browser is not installed
INFO: Epic Privacy Browser browser is not supported
INFO:browser-history:Epic Privacy Browser browser is not supported
INFO: Firefox browser is not installed
INFO:browser-history:Firefox browser is not installed
INFO: LibreWolf browser is not installed
INFO:browser-history:LibreWolf browser is not installed
INFO: Safari browser is not supported
INFO:browser-his

🔍 Last 10 visited URLs:
1.https://www.redtube.com
2.https://example.com/normal-page
3.https://xvideos.com


## sending this to my fastAPI

In [12]:
FASTAPI_URL = "https://6294e6ebe92a.ngrok-free.app"
print("\n🚀 Sending to FastAPI for classification...\n")

for url in latest_urls:
    try:
        response = requests.post(f"{FASTAPI_URL}/check", json={"url": url})
        print(f"🌐 URL: {url}")
        print(f"📦 Raw Response: {response.text}")

        try:
            result = response.json()

            # Check if 'is_adult' key exists
            if 'is_adult' in result:
                status = "❌ Adult" if result["is_adult"] else "✅ Safe"
                print(f"🔗 {url} -> {status}")
            else:
                print(f"⚠️ Missing 'is_adult' key in response for {url}")
                print(f"📝 Full response: {result}")

        except ValueError:
            print(f"🚨 Invalid JSON for {url}: {response.text}")

        time.sleep(1)

    except Exception as e:
        print(f"❌ Error during request for {url}: {e}")


🚀 Sending to FastAPI for classification...

INFO:     34.42.150.82:0 - "POST /check HTTP/1.1" 200 OK
🌐 URL: https://www.redtube.com
📦 Raw Response: {"url":"https://www.redtube.com","is_adult":true}
🔗 https://www.redtube.com -> ❌ Adult
INFO:     34.42.150.82:0 - "POST /check HTTP/1.1" 200 OK
🌐 URL: https://example.com/normal-page
📦 Raw Response: {"url":"https://example.com/normal-page","is_adult":false}
🔗 https://example.com/normal-page -> ✅ Safe
INFO:     34.42.150.82:0 - "POST /check HTTP/1.1" 200 OK
🌐 URL: https://xvideos.com
📦 Raw Response: {"url":"https://xvideos.com","is_adult":true}
🔗 https://xvideos.com -> ❌ Adult
